# 0. Imports & inputs

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from solcore.structure import Junction
from solcore.solar_cell import SolarCell
from solcore.analytic_solar_cells.diode_equation import iv_2diode
from solcore.analytic_solar_cells.IV import iv_multijunction
from solcore.solar_cell_solver import default_options, solar_cell_solver
import pprint
import os
from datetime import datetime
import scipy.constants as const
import pvlib
from scipy.constants import Boltzmann, e

# define the plot template
axis_label_size = 20
axis_num_size = 20
text_size = 20
title_size = 22

# define the figure size for single plot
figure_size = (6, 6)
subplot_size_1_2 = (6, 6)
long_hoz_figsize = (12, 6)
two_by_two_figsize = (12, 12)

# define the random seed for reproducibility
np.random.seed(666)

# define the number of IV curves to simulate and validate
num_curves = 10
# define the module area
module_area = 2 # m2
# define the number of cells in series and parallel
num_s_cells = 72
num_p_cells = 2

# define the range of Rs, Rsh, J01 J02, IL, T for the simulation
Rsmin = 0.1  # Ohm, from the modules I have
Rshmin = 200  # Ohm, from the modules I have
Rsmax = 0.4  # Ohm, from the modules I have
Rshmax = 1200  # Ohm, from the modules I have
J01min = 1e-14  # A/m2, from PV education range
J01max = 1e-8  # A/m2, from PV education range
J02min = 1e-14  # A/m2, from PV education range
J02max = 1e-8  # A/m2, from PV education range
ILmin = 8  # A/m2, from the modules I have
ILmax = 15  # A/m2, from the modules I have
Tmin = 273  # K, from the modules I have
Tmax = 343  # K, from the modules I have
N_param = 1000 # parameter sample size

# print out the default options
print("Default options:")
# Pretty‑print the entire defaults dict:
pprint.pprint(default_options)

# define the folder to save data
save_folder = r"C:\Users\z5183876\Documents\GitHub\24_09_24_Solar_Edge\Results\2_diode_models"

# define the function to calculate the I0 at MPPT
def calc_I0(I, V, Rs, Rsh, n, N, vth):
    # calculate the exponential term
    exp_term = np.exp(-(V + I * Rs) / (n * N * vth))
    # calculate the fraction term
    frac_term = n * N * vth / V
    # calculate hte numerator
    numerator = I*(1 + Rs/Rsh) - V/Rsh
    # calculate the denominator
    denominator = 1 - I * Rs / V
    # put them together
    I0 = numerator / denominator * frac_term * exp_term
    return I0

# define the function to calculate IL at MPPT
def calc_IL(I, V, Rs, Rsh, n, N, vth, I0):
    # calculate the first term
    first_term = I * (1 + Rs/Rsh)
    # calculate the second term
    second_term = V / Rsh
    # calculate the third term
    third_term = I0*(np.exp((V + I * Rs) / (n * N * vth)) - 1)
    # put them together
    IL = first_term + second_term + third_term
    return IL

# 1. Parameter simulation

In [ ]:
# Generate your parameter distributions (linear space)
for param in ['Rs', 'Rsh', 'IL', 'T']:
    mean     = (eval(param + 'max') + eval(param + 'min')) / 2
    std      = (eval(param + 'max') - eval(param + 'min')) / 6
    globals()[param + '_values'] = np.random.normal(mean, std, N_param)

# Special handling for J01 and J02: sample log-values (as before)
for param in ['J01', 'J02']:
    log_min   = np.log(eval(param + 'min'))
    log_max   = np.log(eval(param + 'max'))
    mean_log  = (log_max + log_min) / 2
    std_log   = (log_max - log_min) / 6
    log_vals  = np.random.normal(mean_log, std_log, N_param)
    globals()[param + '_values'] = np.exp(log_vals)

# Plotting all six on a 2×3 grid of subplots, 
# but for J01/J02 we plot their natural log
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for ax, param in zip(axes, ['Rs', 'Rsh', 'J01', 'J02', 'IL', 'T']):
    values = globals()[param + '_values']
    if param in ['J01', 'J02']:
        # take natural log for plotting
        values = np.log(values)
        xlabel = f'log({param})'
        title  = f'Distribution of log({param})'
    else:
        xlabel = param
        title  = f'Distribution of {param}'
    
    ax.hist(values,
            bins=20,
            density=True,
            alpha=0.6,
            color='g')
    ax.set_title(title, fontsize=title_size)
    ax.set_xlabel(xlabel, fontsize=axis_label_size)
    ax.set_ylabel('Density', fontsize=axis_label_size)
    ax.tick_params(axis='both', labelsize=axis_num_size)

plt.tight_layout()
plt.show()

# resample num_curves number of times from the distributions, create a DataFrame
# Draw num_curves samples from each _values array
samples = {}
for param in ['Rs', 'Rsh', 'J01', 'J02', 'IL', 'T']:
    samples[param] = np.random.choice(
        globals()[param + '_values'],
        size=num_curves,
        replace=True
    )

# Build DataFrame
df = pd.DataFrame(samples)

# print the head
print(df)


# 2. Two diode model IV curve generation

## 2.1. With Rs

In [ ]:
# %%
for index, row in df.iterrows():
    # Unpack
    Rs, Rsh = row['Rs'], row['Rsh']
    J01, J02 = row['J01'], row['J02']        # A/cm²
    IL       = row['IL']                     # A
    T        = row['T']                      # K
    module_ID = f"{index+1}"

    # Half‑cell area
    A_hcell_m2  = module_area / (num_s_cells*num_p_cells)     # m²
    A_hcell_cm2 = A_hcell_m2 * 1e4                            # cm²

    # Build 2‑diode junction
    junction = Junction(
        kind='2D', T=T,
        j01=J01, j02=J02, n1=1.0, n2=2.0,
        R_shunt = (Rsh/num_s_cells*num_p_cells)*A_hcell_cm2,
        jsc     = (IL/num_p_cells)/A_hcell_cm2,
        A       = A_hcell_cm2
    )
    opts = default_options.copy()
    opts.update({
        'internal_voltages': np.linspace(0, 0.8, 10000),
        'light_iv': True, 'T': T, 'mpp': True
    })
    iv_2diode(junction, opts)

    # build the cell object
    cell = SolarCell(
        [junction],
        R_series=Rs, R_shunt=Rsh, T=T,
        A=A_hcell_cm2,
    )
    # solve the cell IV
    # iv_multijunction(cell, opts)
    solar_cell_solver(cell, 'iv', opts)

    # extract the IV data
    V_cell, I_cell = cell.iv['IV']


    # Extract IV arrays
    V_module  = (V_cell * num_s_cells)       # V
    I_module  = (I_cell*num_p_cells)*A_hcell_cm2                  # A
    mask  = I_module > -0.1
    V_module = V_module[mask]
    I_module = I_module[mask]


    # Performance metrics
    I_sc = I_module[0]
    V_oc = V_module[np.where(I_module > 0)[0][-1]]
    P    = V_module * I_module
    idx  = np.argmax(P)
    P_max= P[idx]
    V_mp = V_module[idx]
    I_mp = I_module[idx]
    FF   = P_max/(V_oc*I_sc)

    # --- Scalar n extraction (including Rsh) ---
    Vt = Boltzmann * T / e
    # Convert J0 densities from A/cm² → A/m²
    J01_SI = J01 * 1e4
    J02_SI = J02 * 1e4
    # Module-level I0 (A)
    I0_module = (J01_SI + J02_SI) * A_hcell_m2 * num_p_cells
    # I0_module = 1e-90
    # Use I_ph ≈ I_sc
    numerator = I_sc - V_oc/Rsh
    n_scalar  = V_oc / (Vt * np.log(numerator/I0_module + 1))/num_s_cells

    # Plot and table
    fig, (ax_curve, ax_table) = plt.subplots(
        1, 2,
        figsize=(figure_size[0]*2, figure_size[1]),
        gridspec_kw={'width_ratios': [3, 1]}
    )
    ax_curve.plot(V_module, I_module, label='IV curve')
    ax_curve.plot(V_mp, I_mp, 'o',
                  markeredgecolor='red', markerfacecolor='red',
                  markersize=8, label='MPP')
    ax_curve.set_xlabel('Voltage (V)', fontsize=axis_label_size)
    ax_curve.set_ylabel('Current (A)', fontsize=axis_label_size)
    ax_curve.set_title('Module ID ' + module_ID, fontsize=title_size)
    ax_curve.tick_params(labelsize=axis_num_size)
    ax_curve.grid(True)
    ax_curve.legend(fontsize=text_size, loc='lower left', bbox_to_anchor=(0.1,0.1), frameon=False)

    # Table data (now including n_scalar)
    table_data = [
        [f'{J01:.2e}',   'A/cm²'],
        [f'{J02:.2e}',   'A/cm²'],
        [f'{Rs:.2f}',    'Ω'],
        [f'{Rsh:.2f}',   'Ω'],
        [f'{IL:.2f}',    'A'],
        [f'{T:.1f}',     'K'],
        [f'{I_sc:.2f}',  'A'],
        [f'{V_oc:.2f}',  'V'],
        [f'{P_max:.2f}', 'W'],
        [f'{V_mp:.2f}',  'V'],
        [f'{I_mp:.2f}',  'A'],
        [f'{FF:.3f}',    '–'],
        [f'{n_scalar:.3f}','–']
    ]
    row_labels = [
        r'$J_{01}$', r'$J_{02}$', r'$R_{s}$',  r'$R_{sh}$',
        r'$I_{L}$',  r'$T$',      r'$I_{sc}$', r'$V_{oc}$',
        r'$P_{max}$',r'$V_{mp}$', r'$I_{mp}$', r'$FF$',
        r'$n$'
    ]
    col_labels = ['Value','Unit']

    ax_table.axis('off')
    tbl = ax_table.table(
        cellText=table_data,
        rowLabels=row_labels,
        colLabels=col_labels,
        loc='center'
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(14)
    tbl.scale(1, 1.5)

    plt.tight_layout()

    # Save to timestamped folder
    timestamp    = datetime.now().strftime("%Y%m%d_%H%M%S")
    curve_folder = os.path.join(save_folder, timestamp)
    os.makedirs(curve_folder, exist_ok=True)

    fig.savefig(os.path.join(curve_folder, "IV_curve.png"), dpi=300, bbox_inches='tight')
    plt.show()

    pd.DataFrame(table_data, columns=col_labels, index=row_labels).to_csv(
        os.path.join(curve_folder, "IV_curve_table.csv"),
        index=True, header=True
    )
    pd.DataFrame({'V': V_module, 'I': I_module}).to_csv(
        os.path.join(curve_folder, "IV_curve_data.csv"),
        index=False
    )

## 2.2. Without Rs

In [ ]:
# # %%
# for index, row in df.iterrows():
#     # Unpack
#     Rs, Rsh = row['Rs'], row['Rsh']
#     J01, J02 = row['J01'], row['J02']        # A/cm²
#     IL       = row['IL']                     # A/m²
#     T        = row['T']                      # K

#     # Half‑cell area
#     A_hcell_m2  = module_area / (num_s_cells*num_p_cells)     # m²
#     A_hcell_cm2 = A_hcell_m2 * 1e4                            # cm²

#     # Build 2‑diode junction
#     junction = Junction(
#         kind='2D', T=T,
#         j01=J01, j02=J02, n1=1.0, n2=2.0,
#         R_shunt = (Rsh/num_s_cells*num_p_cells)*A_hcell_cm2,
#         jsc     = (IL/num_p_cells)/A_hcell_cm2,
#         A       = A_hcell_cm2
#     )
#     opts = default_options.copy()
#     opts.update({
#         'internal_voltages': np.linspace(0, 0.8, 2000),
#         'light_iv': True, 'T': T, 'mpp': True
#     })
#     iv_2diode(junction, opts)

#     # Extract IV arrays
#     V = opts['internal_voltages']
#     J = junction.iv(V)
#     I_mod = -J * A_hcell_cm2 * num_p_cells   # A
#     mask  = I_mod > -0.1
#     V_module  = V[mask] * num_s_cells        # V
#     I_module  = I_mod[mask]                  # A

#     # Performance metrics
#     I_sc = I_module[0]
#     V_oc = V_module[np.where(I_module > 0)[0][-1]]
#     P    = V_module * I_module
#     idx  = np.argmax(P)
#     P_max= P[idx]
#     V_mp = V_module[idx]
#     I_mp = I_module[idx]
#     FF   = P_max/(V_oc*I_sc)

#     # --- Scalar n extraction (including Rsh) ---
#     Vt = Boltzmann * T / e
#     # Convert J0 densities from A/cm² → A/m²
#     J01_SI = J01 * 1e4
#     J02_SI = J02 * 1e4
#     # Module-level I0 (A)
#     I0_module = (J01_SI + J02_SI) * A_hcell_m2 * num_p_cells
#     # I0_module = 1e-90
#     # Use I_ph ≈ I_sc
#     numerator = I_sc - V_oc/Rsh
#     n_scalar  = V_oc / (Vt * np.log(numerator/I0_module + 1))/num_s_cells

#     # Plot and table
#     fig, (ax_curve, ax_table) = plt.subplots(
#         1, 2,
#         figsize=(figure_size[0]*2, figure_size[1]),
#         gridspec_kw={'width_ratios': [3, 1]}
#     )
#     ax_curve.plot(V_module, I_module, label='IV curve')
#     ax_curve.plot(V_mp, I_mp, 'o',
#                   markeredgecolor='red', markerfacecolor='red',
#                   markersize=8, label='MPP')
#     ax_curve.set_xlabel('Voltage (V)', fontsize=axis_label_size)
#     ax_curve.set_ylabel('Current (A)', fontsize=axis_label_size)
#     ax_curve.set_title('2‑Diode IV Curve', fontsize=title_size)
#     ax_curve.tick_params(labelsize=axis_num_size)
#     ax_curve.grid(True)
#     ax_curve.legend(fontsize=text_size, loc='lower left', bbox_to_anchor=(0.1,0.1), frameon=False)

#     # Table data (now including n_scalar)
#     table_data = [
#         [f'{J01:.2e}',   'A/cm²'],
#         [f'{J02:.2e}',   'A/cm²'],
#         [f'{Rs:.2f}',    'Ω'],
#         [f'{Rsh:.2f}',   'Ω'],
#         [f'{IL:.2f}',    'A/m²'],
#         [f'{T:.1f}',     'K'],
#         [f'{I_sc:.2f}',  'A'],
#         [f'{V_oc:.2f}',  'V'],
#         [f'{P_max:.2f}', 'W'],
#         [f'{V_mp:.2f}',  'V'],
#         [f'{I_mp:.2f}',  'A'],
#         [f'{FF:.3f}',    '–'],
#         [f'{n_scalar:.3f}','–']
#     ]
#     row_labels = [
#         r'$J_{01}$', r'$J_{02}$', r'$R_{s}$',  r'$R_{sh}$',
#         r'$I_{L}$',  r'$T$',      r'$I_{sc}$', r'$V_{oc}$',
#         r'$P_{max}$',r'$V_{mp}$', r'$I_{mp}$', r'$FF$',
#         r'$n$'
#     ]
#     col_labels = ['Value','Unit']

#     ax_table.axis('off')
#     tbl = ax_table.table(
#         cellText=table_data,
#         rowLabels=row_labels,
#         colLabels=col_labels,
#         loc='center'
#     )
#     tbl.auto_set_font_size(False)
#     tbl.set_fontsize(14)
#     tbl.scale(1, 1.5)

#     plt.tight_layout()

#     # Save to timestamped folder
#     timestamp    = datetime.now().strftime("%Y%m%d_%H%M%S")
#     curve_folder = os.path.join(save_folder, timestamp)
#     os.makedirs(curve_folder, exist_ok=True)

#     fig.savefig(os.path.join(curve_folder, "IV_curve.png"), dpi=300, bbox_inches='tight')
#     plt.show()

#     pd.DataFrame(table_data, columns=col_labels, index=row_labels).to_csv(
#         os.path.join(curve_folder, "IV_curve_table.csv"),
#         index=True, header=True
#     )
#     pd.DataFrame({'V': V_module, 'I': I_module}).to_csv(
#         os.path.join(curve_folder, "IV_curve_data.csv"),
#         index=False
#     )

# 3. IV curve reconstruction using given method

## 3.1. IV reconstruction

In [ ]:
# for each folder in save_folder check if there is a file containing "IV_curve_reconstructed"
if os.path.exists(save_folder):
    for folder in os.listdir(save_folder):
        folder_path = os.path.join(save_folder, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if "IV_curve_reconstructed" not in file:
                    # read the csv file that contains word table
                    if "IV_curve_table" in file:
                        # read the table
                        table_df = pd.read_csv(os.path.join(folder_path, file))
                        # for the first column, each row, remove any symbols including: $, _, {, }
                        table_df.iloc[:, 0] = table_df.iloc[:, 0].str.replace(r'[$_{}]', '', regex=True)
                        print(table_df)
                        # read the Rs, Rsh, n, Vmp, Imp from the table
                        Rs = table_df.iloc[2, 1]
                        Rsh = table_df.iloc[3, 1]
                        n = 1 # n is not in the table, but we can assume it is 1 for this case
                        Vmp = table_df.iloc[9, 1]
                        Imp = table_df.iloc[10, 1]
                        # read the T
                        T = table_df.iloc[5, 1]
                        # read ideality factor n from the table
                        n = table_df.iloc[12, 1]

                        # calculate the required parameters
                        # calculate the thermal voltage
                        k = const.Boltzmann
                        q = const.e
                        vth = k * T / q
                        # calculate the I0
                        I0_value = calc_I0(Imp, Vmp, Rs, Rsh, n, num_s_cells, vth)
                        # calculate the IL
                        IL_value = calc_IL(Imp, Vmp, Rs, Rsh, n, num_s_cells, vth, I0_value)

                        # set up the IV curves for pvlib
                        params = {
                            'photocurrent': IL_value,
                            'saturation_current': I0_value,
                            'resistance_series': Rs,
                            'resistance_shunt': Rsh,
                            'nNsVth': n * num_s_cells * vth,
                            }
                        
                        # compute the IV curve
                        # find the v that was used in the 2 diode model IV curve, read the file 'IV_curve_data.csv' in the file
                        iv_curve_df = pd.read_csv(os.path.join(folder_path, "IV_curve_data.csv"))
                        # extract the V and I from the dataframe
                        V_2_diode = iv_curve_df['V'].values
                        I_2_diode = iv_curve_df['I'].values
                        # # compute the reconstructed IV
                        # V_reconstructed = pvlib.pvsystem.v_from_i(
                        #     current=I_2_diode,
                        #     photocurrent=IL_value,
                        #     saturation_current=I0_value,
                        #     resistance_series=Rs,
                        #     resistance_shunt=Rsh,
                        #     nNsVth=n * num_s_cells * vth,
                        #     method='lambertw'
                        # )
                        # I_reconstructed = I_2_diode.copy()
                        V_reconstructed = V_2_diode.copy()
                        I_reconstructed = pvlib.pvsystem.i_from_v(
                            voltage=V_reconstructed,
                            photocurrent=IL_value,
                            saturation_current=I0_value,
                            resistance_series=Rs,
                            resistance_shunt=Rsh,
                            nNsVth=n * num_s_cells * vth,
                            method='lambertw'
                        )

                        # calculate the rmse between I_2_diode and I_reconstructed
                        rmse = np.sqrt(np.mean((I_2_diode - I_reconstructed)**2))
                        # calculate hte MAE between I_2_diode and I_reconstructed
                        mae = np.mean(np.abs(I_2_diode - I_reconstructed))
                        # calculate the MAPE between I_2_diode and I_reconstructed
                        mape = np.mean(np.abs((I_2_diode - I_reconstructed) / I_2_diode)) * 100
                        # calculate the R2 between I_2_diode and I_reconstructed
                        ss_res = np.sum((I_2_diode - I_reconstructed)**2)
                        ss_tot = np.sum((I_2_diode - np.mean(I_2_diode))**2)
                        r2 = 1 - (ss_res / ss_tot)
                        # print the results
                        print("RMSE: ", rmse)
                        print("MAE: ", mae)
                        print("MAPE: ", mape, '%')
                        print("R2: ", r2)

                        # plot the reconstructed IV curve with the original IV curve
                        plt.figure(figsize=figure_size)
                        plt.plot(V_2_diode, I_2_diode, label='2 diode model IV')
                        plt.plot(V_reconstructed, I_reconstructed, label='Reconstructed single diode IV')
                        plt.scatter(Vmp, Imp, color='red', label='Maximum power point')
                        plt.xlabel('Voltage (V)', fontsize=axis_label_size)
                        plt.ylabel('Current (A)', fontsize=axis_label_size)
                        plt.title('MAE=' + str(round(mae, 3)) + 'A', fontsize=title_size)    
                        plt.tick_params(axis='both', labelsize=axis_num_size)
                        plt.grid(True)
                        plt.legend(fontsize=text_size-5, loc='lower left', bbox_to_anchor=(0.1, 0.1), frameon=False)
                        plt.tight_layout()
                        plt.show()



                        

## 3.2. Evaluation matrix summary

In [ ]:
# Initialize lists to collect metrics
rmse_list = []
mae_list  = []
mape_list = []
r2_list   = []
J02_list = []
J01_list = []

# Loop through each curve folder
for folder in os.listdir(save_folder):
    folder_path = os.path.join(save_folder, folder)
    if not os.path.isdir(folder_path):
        continue

    # 1) Read the IV data
    iv_df = pd.read_csv(os.path.join(folder_path, "IV_curve_data.csv"))
    V_orig = iv_df['V'].values
    I_orig = iv_df['I'].values

    # 2) Read and clean the parameter table
    table_file = next(f for f in os.listdir(folder_path) if "IV_curve_table" in f)
    table_df   = pd.read_csv(os.path.join(folder_path, table_file))
    table_df.iloc[:, 0] = table_df.iloc[:, 0].str.replace(r'[$_{}]', '', regex=True)
    # print(table_df)

    # Extract parameters (as floats)
    Rs   = float(table_df.iloc[2, 1])
    Rsh  = float(table_df.iloc[3, 1])
    IL_mod = float(table_df.iloc[4, 1])
    T    = float(table_df.iloc[5, 1])
    Vmp  = float(table_df.iloc[9, 1])
    Imp  = float(table_df.iloc[10, 1])
    n    = 1  # assumed
    J01 = float(table_df.iloc[0, 1])
    J02 = float(table_df.iloc[1, 1])

    # Thermal voltage
    vth =k * T / q

    # Compute I0 and IL for single-diode reconstruction
    I0_val = calc_I0(Imp, Vmp, Rs, Rsh, n, num_s_cells, vth)
    IL_val = calc_IL(Imp, Vmp, Rs, Rsh, n, num_s_cells, vth, I0_val)

    # 3) Reconstruct with PVlib
    I_rec = pvlib.pvsystem.i_from_v(
        voltage=V_orig,
        photocurrent=IL_val,
        saturation_current=I0_val,
        resistance_series=Rs,
        resistance_shunt=Rsh,
        nNsVth=n * num_s_cells * vth,
        method='lambertw'
    )

    # 4) Calculate metrics
    diff   = I_rec - I_orig
    rmse   = np.sqrt(np.mean(diff**2))
    mae    = np.mean(np.abs(diff))
    mape   = np.mean(np.abs(diff / I_orig)) * 100
    ss_res = np.sum(diff**2)
    ss_tot = np.sum((I_orig - np.mean(I_orig))**2)
    r2     = 1 - ss_res/ss_tot

    # 5) Store
    rmse_list.append(rmse)
    mae_list.append(mae)
    mape_list.append(mape)
    r2_list.append(r2)
    J02_list.append(J02)
    J01_list.append(J01)

# 6) Plot 2×2 histogram of the metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].hist(rmse_list, bins=50, edgecolor='black')
axes[0, 0].set_title('RMSE Distribution')
axes[0, 0].set_xlabel('RMSE (A)')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].hist(mae_list, bins=50, edgecolor='black')
axes[0, 1].set_title('MAE Distribution')
axes[0, 1].set_xlabel('MAE (A)')
axes[0, 1].set_ylabel('Frequency')

axes[1, 0].hist(mape_list, bins=50, edgecolor='black')
axes[1, 0].set_title('MAPE Distribution')
axes[1, 0].set_xlabel('MAPE (%)')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(r2_list, bins=50, edgecolor='black')
axes[1, 1].set_title('R² Distribution')
axes[1, 1].set_xlabel('R²')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


metrics = {
    'RMSE (A)': rmse_list,
    'MAE (A)':  mae_list,
    'MAPE (%)': mape_list,
    'R²':        r2_list
}

for name, data in metrics.items():
    fig, ax = plt.subplots(figsize=figure_size)

    # Plot histogram
    ax.hist(data, bins=50, edgecolor='black')
    ax.set_title(f'{name} Distribution', fontsize=title_size)
    ax.set_xlabel(name, fontsize=axis_label_size)
    ax.set_ylabel('Frequency', fontsize=axis_label_size)
    ax.tick_params(axis='both', labelsize=axis_num_size)

    # Compute mean and median
    mean_val   = np.mean(data)
    median_val = np.median(data)

    # Draw vertical dashed lines
    ax.axvline(mean_val,   color='red',   linestyle='--', linewidth=2)
    ax.axvline(median_val, color='black', linestyle='--', linewidth=2)

    # Annotate mean & median in top‑right
    ax.text(
        0.95, 0.95,
        f"Mean = {mean_val:.3f}\nMedian = {median_val:.3f}",
        transform=ax.transAxes,
        ha='right', va='top',
        fontsize=text_size-2,
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
    )

    plt.tight_layout()
    plt.show()

# also create a plot between each evaluation matrix vs J02
for name, data in metrics.items():
    fig, ax = plt.subplots(figsize=figure_size)

    ax.scatter(np.log(J02_list), data, edgecolor='black')
    ax.set_title(f'{name} vs J$_{{02}}$', fontsize=title_size)
    ax.set_xlabel(r'J$_{02}$ (A/cm²)', fontsize=axis_label_size)
    ax.set_ylabel(name,            fontsize=axis_label_size)
    ax.tick_params(axis='both',    labelsize=axis_num_size)
    ax.grid(True)

    # Optional: annotate Pearson R
    # r = np.corrcoef(J02_list, data)[0,1]
    # ax.text(0.05, 0.95, f"r = {r:.2f}", transform=ax.transAxes,
    #         ha='left', va='top', fontsize=text_size)

    plt.tight_layout()
    plt.show()